In [1]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
# Use ensemble to train the model
from sklearn.ensemble import VotingRegressor
from sklearn.model_selection import cross_val_score
import numpy as np
import pandas as pd
import lightgbm as lgb
import sys
import sys

# In the second part of the notebook
from sklearn.preprocessing import LabelEncoder

# This part of importing would be moved to main.py or train.py later
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import lightgbm as lgb

from scipy.stats import zscore
from scipy.special import boxcox1p

import pandas as pd
from sklearn.feature_selection import chi2
from sklearn.preprocessing import LabelEncoder
# from catboost import CatBoostRegressor

import lightgbm as lgb
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold

import lightgbm as lgb
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
import wandb

Define a hyperparameter class 

In [ ]:
class Config:
    def __init__(self):
        self.objective = 'regression'
        self.num_leaves = 31
        self.learning_rate = 0.01
        self.n_estimators = 120
        self.max_bin = 255
        self.bagging_fraction = 0.8
        self.bagging_freq = 5
        self.feature_fraction = 0.8
        self.feature_fraction_seed = 9
        self.bagging_seed = 9
        self.random_state = 42
        self.OrgFertilizers = 'None'
        self.FirstTopDressFert = 'None' 
        self.CropbasalFerts = 'None'
        self.NursDetFactor = 'None'
        self.LandPreparationMethod = 'None'
        self.TransDetFactor = 'None'
        self.PCropSolidOrgFertAppMethod = 'None'
        self.sparse_threshold = 1.0
# Instantiate the Hyperparameters class
config = Config()

# Initialize a wandb run
wandb.init(project='zindi-crop-challenge',
           entity="lmu-seminar")

# Update the wandb configuration with the hyperparameters
wandb.config.update(vars(config))
    

In [2]:
# Read the dataset
path = 'Train.csv'
dataset = pd.read_csv(path)

In [3]:
test_path = 'Test.csv'
dataset_test = pd.read_csv(test_path)

In [4]:
# Save the 'Yield' column
train_labels = dataset['Yield'].copy()
dataset = dataset.drop(columns=['ID','Yield'], axis = 1)
ID = dataset_test['ID'].copy()
dataset_test = dataset_test.drop(columns=['ID'], axis = 1)

# Copy the original dataset
dataset_original = dataset.copy()
dataset_test_original = dataset_test.copy()

In [5]:
dataset['Harv_hand_rent'] = dataset['Harv_hand_rent'].fillna(0)
dataset['OrgFertilizers'] = dataset['OrgFertilizers'].fillna(dataset['OrgFertilizers'].mode()[0])
dataset['CropbasalFerts'] = dataset['CropbasalFerts'].fillna(dataset['CropbasalFerts'].mode()[0])
dataset['FirstTopDressFert'] = dataset['FirstTopDressFert'].fillna(dataset['FirstTopDressFert'].mode()[0])
dataset['NursDetFactor'] = dataset['NursDetFactor'].fillna(dataset['NursDetFactor'].mode()[0])
dataset['LandPreparationMethod'] = dataset['LandPreparationMethod'].fillna(dataset['LandPreparationMethod'].mode()[0])
dataset['TransDetFactor'] = dataset['TransDetFactor'].fillna(dataset['TransDetFactor'].mode()[0])
dataset['PCropSolidOrgFertAppMethod'] = dataset['PCropSolidOrgFertAppMethod'].fillna(dataset['PCropSolidOrgFertAppMethod'].mode()[0])

dataset_test['Harv_hand_rent'] = dataset_test['Harv_hand_rent'].fillna(0)
dataset_test['OrgFertilizers'] = dataset_test['OrgFertilizers'].fillna(dataset_test['OrgFertilizers'].mode()[0])
dataset_test['CropbasalFerts'] = dataset_test['CropbasalFerts'].fillna(dataset_test['CropbasalFerts'].mode()[0])
dataset_test['FirstTopDressFert'] = dataset_test['FirstTopDressFert'].fillna(dataset_test['FirstTopDressFert'].mode()[0])
dataset_test['NursDetFactor'] = dataset_test['NursDetFactor'].fillna(dataset_test['NursDetFactor'].mode()[0])
dataset_test['LandPreparationMethod'] = dataset_test['LandPreparationMethod'].fillna(dataset_test['LandPreparationMethod'].mode()[0])
dataset_test['TransDetFactor'] = dataset_test['TransDetFactor'].fillna(dataset_test['TransDetFactor'].mode()[0])
dataset_test['PCropSolidOrgFertAppMethod'] = dataset_test['PCropSolidOrgFertAppMethod'].fillna(dataset_test['PCropSolidOrgFertAppMethod'].mode()[0])

In [6]:
# Checking the unique values for categorical variables to identify sparse classes
categorical_columns = dataset.select_dtypes(include=['object']).columns
sparse_classes = {col: dataset[col].nunique() for col in categorical_columns if dataset[col].nunique() > 15} 

In [7]:
# check the name of sparse classes
print(sparse_classes.keys())

dict_keys(['LandPreparationMethod', 'CropTillageDate', 'RcNursEstDate', 'SeedingSowingTransplanting', 'NursDetFactor', 'TransDetFactor', 'OrgFertilizers', 'CropbasalFerts', 'Harv_date', 'Threshing_date'])


In [8]:
# check the unique values of sparse classes
print(sparse_classes.values())

dict_values([43, 78, 61, 62, 125, 155, 31, 34, 107, 162])


In [9]:
# Define a threshold for grouping
threshold_percentage = 1
threshold = len(dataset) * (threshold_percentage / 100)

In [10]:
# Function to group sparse classes
def group_sparse_classes(df, column, threshold):
    # Find the categories that are below the threshold
    value_counts = df[column].value_counts()
    to_replace = value_counts[value_counts <= threshold].index.tolist()
    
    # Replace the sparse classes with 'other'
    df[column] = df[column].replace(to_replace, 'other')
    return df

In [11]:
# Apply grouping for the identified categorical variables with many unique values
for col in ['LandPreparationMethod', 'OrgFertilizers', 'CropbasalFerts']:
    dataset = group_sparse_classes(dataset, col, threshold)

In [12]:
# Apply the category adjustment to the test dataset based on the training dataset categories
for col in ['LandPreparationMethod', 'OrgFertilizers', 'CropbasalFerts']:
    test_dataset = group_sparse_classes(dataset_test, col, threshold)

In [13]:
# fill the high skewness columns with median
# check the skewness of the numerical columns
skewness = dataset.skew(numeric_only=True)
skewness = skewness[abs(skewness) > 0.5]
print("There are {} skewed numerical features to Box Cox transform".format(skewness.shape[0]))

# sort the skewness in descending order
skewness.sort_values(ascending=False)
print(skewness)

There are 18 skewed numerical features to Box Cox transform
CultLand                         8.178796
CropCultLand                     8.796317
SeedlingsPerPit                 53.528364
TransplantingIrrigationHours    30.754264
TransIrriCost                    3.912326
StandingWater                    1.848641
Ganaura                          5.237242
CropOrgFYM                       6.876849
BasalDAP                         2.905715
BasalUrea                        2.244465
1tdUrea                          2.094479
1appDaysUrea                     4.769872
2tdUrea                          2.696580
2appDaysUrea                    -1.589283
Harv_hand_rent                  40.547049
Residue_length                  -1.880371
Residue_perc                     3.747956
Acre                             2.384037
dtype: float64


In [14]:
high_skewed_features = ['SeedlingsPerPit', 'TransplantingIrrigationHours','Harv_hand_rent']
low_skewed_features = ['CultLand', 'CropOrgFYM', 'CropCultLand','Ganaura'] 
moderate_skewed_features = ['BasalUrea', 'BasalDAP',
                            '1tdUrea', '1appDaysUrea', 
                            '2tdUrea', '2appDaysUrea',
                            'Residue_length', 'Residue_perc',
                            'StandingWater', 'TransIrriCost',
                            'TransIrriCost']

In [15]:
# check the dtype of those skewed features
print(dataset[high_skewed_features].dtypes)
print(dataset[low_skewed_features].dtypes)
print(dataset[moderate_skewed_features].dtypes)

SeedlingsPerPit                 float64
TransplantingIrrigationHours    float64
Harv_hand_rent                  float64
dtype: object
CultLand          int64
CropOrgFYM      float64
CropCultLand      int64
Ganaura         float64
dtype: object
BasalUrea         float64
BasalDAP          float64
1tdUrea           float64
1appDaysUrea      float64
2tdUrea           float64
2appDaysUrea      float64
Residue_length      int64
Residue_perc        int64
StandingWater     float64
TransIrriCost     float64
TransIrriCost     float64
dtype: object


In [16]:
# Fill the low skewness columns with mean
for feat in low_skewed_features:
    dataset[feat] = dataset[feat].fillna(dataset[feat].mean())
    dataset_test[feat] = dataset_test[feat].fillna(dataset_test[feat].mean())

# Fill the moderate skewness columns with median
for feat in moderate_skewed_features:
    dataset[feat] = dataset[feat].fillna(dataset[feat].median())
    dataset_test[feat] = dataset_test[feat].fillna(dataset_test[feat].median())

# Fill the high skewness columns with median
for feat in high_skewed_features:
    dataset[feat] = dataset[feat].fillna(dataset[feat].median())
    dataset_test[feat] = dataset_test[feat].fillna(dataset_test[feat].median())

In [17]:
"""
skewed_features = skewness.index
lam = 0.15
# apply the boxcox1p transformation to the skewed features
for feat in skewed_features:
    dataset[feat] = boxcox1p(dataset[feat], lam)
    dataset_test[feat] = boxcox1p(dataset_test[feat], lam)

# check the skewness of the numerical columns
skewness = dataset.skew(numeric_only=True)
skewness = skewness[abs(skewness) > 0.5]
print("There are {} skewed numerical features to Box Cox transform".format(skewness.shape[0]))
"""

'\nskewed_features = skewness.index\nlam = 0.15\n# apply the boxcox1p transformation to the skewed features\nfor feat in skewed_features:\n    dataset[feat] = boxcox1p(dataset[feat], lam)\n    dataset_test[feat] = boxcox1p(dataset_test[feat], lam)\n\n# check the skewness of the numerical columns\nskewness = dataset.skew(numeric_only=True)\nskewness = skewness[abs(skewness) > 0.5]\nprint("There are {} skewed numerical features to Box Cox transform".format(skewness.shape[0]))\n'

In [18]:
"""
bins_SeedlingsPerPit = [0, 2, 4, np.inf]
labels_SeedlingsPerPit = ['Low', 'Medium', 'High']
dataset['SeedlingsPerPit_Binned'] = pd.cut(dataset['SeedlingsPerPit'], bins=bins_SeedlingsPerPit, labels=labels_SeedlingsPerPit)
dataset_test['SeedlingsPerPit_Binned'] = pd.cut(dataset_test['SeedlingsPerPit'], bins=bins_SeedlingsPerPit, labels=labels_SeedlingsPerPit)
"""

"\nbins_SeedlingsPerPit = [0, 2, 4, np.inf]\nlabels_SeedlingsPerPit = ['Low', 'Medium', 'High']\ndataset['SeedlingsPerPit_Binned'] = pd.cut(dataset['SeedlingsPerPit'], bins=bins_SeedlingsPerPit, labels=labels_SeedlingsPerPit)\ndataset_test['SeedlingsPerPit_Binned'] = pd.cut(dataset_test['SeedlingsPerPit'], bins=bins_SeedlingsPerPit, labels=labels_SeedlingsPerPit)\n"

In [19]:
"""
bins_NoFertilizerAppln = [0, 1, 2, np.inf]
labels_NoFertilizerAppln = ['Low', 'Medium', 'High']
dataset['NoFertilizerAppln_Binned'] = pd.cut(dataset['NoFertilizerAppln'], bins=bins_NoFertilizerAppln, labels=labels_NoFertilizerAppln)
dataset_test['NoFertilizerAppln_Binned'] = pd.cut(dataset_test['NoFertilizerAppln'], bins=bins_NoFertilizerAppln, labels=labels_NoFertilizerAppln)
"""

"\nbins_NoFertilizerAppln = [0, 1, 2, np.inf]\nlabels_NoFertilizerAppln = ['Low', 'Medium', 'High']\ndataset['NoFertilizerAppln_Binned'] = pd.cut(dataset['NoFertilizerAppln'], bins=bins_NoFertilizerAppln, labels=labels_NoFertilizerAppln)\ndataset_test['NoFertilizerAppln_Binned'] = pd.cut(dataset_test['NoFertilizerAppln'], bins=bins_NoFertilizerAppln, labels=labels_NoFertilizerAppln)\n"

In [20]:
# if RcNursEstDate is NaN, then fill NursDetFactor with None
# if RcNursEstDate is not NaN, then fill NursDetFactor with mode
dataset.loc[dataset['RcNursEstDate'].isnull(), 'NursDetFactor'] = 'None'
dataset.loc[dataset['RcNursEstDate'].notnull(), 'NursDetFactor'] = dataset['NursDetFactor'].mode()[0]

dataset_test.loc[dataset_test['RcNursEstDate'].isnull(), 'NursDetFactor'] = 'None'
dataset_test.loc[dataset_test['RcNursEstDate'].notnull(), 'NursDetFactor'] = dataset_test['NursDetFactor'].mode()[0]

In [21]:
# Fill the missing value with MineralFertAppMethod.1 with mode
dataset['MineralFertAppMethod.1'] = dataset['MineralFertAppMethod.1'].fillna(dataset['MineralFertAppMethod.1'].mode()[0])
dataset_test['MineralFertAppMethod.1'] = dataset_test['MineralFertAppMethod.1'].fillna(dataset_test['MineralFertAppMethod.1'].mode()[0])

In [22]:
# date columns
date_columns = ['Harv_date', 'SeedingSowingTransplanting', 'RcNursEstDate', 'Threshing_date', 'CropTillageDate']

# fill the missing values with mode
for col in date_columns:
    dataset[col] = dataset[col].fillna(dataset[col].mode()[0])
    dataset_test[col] = dataset_test[col].fillna(dataset_test[col].mode()[0])

In [23]:
# convert the date columns to datetime
for col in date_columns:
    dataset[col] = pd.to_datetime(dataset[col])
    dataset_test[col] = pd.to_datetime(dataset_test[col])

In [24]:
dataset['Harv_date_RcNursEstDate'] = (dataset['Harv_date'] - dataset['RcNursEstDate']).dt.days
dataset_test['Harv_date_RcNursEstDate'] = (dataset_test['Harv_date'] - dataset_test['RcNursEstDate']).dt.days
dataset.loc[dataset['Harv_date_RcNursEstDate'] < 0, 'Harv_date_RcNursEstDate'] = dataset.loc[dataset['Harv_date_RcNursEstDate'] < 0, 'Harv_date_RcNursEstDate'] + 365
dataset_test.loc[dataset_test['Harv_date_RcNursEstDate'] < 0, 'Harv_date_RcNursEstDate'] = dataset_test.loc[dataset_test['Harv_date_RcNursEstDate'] < 0, 'Harv_date_RcNursEstDate'] + 365

In [25]:
# Harvest_date - CropTillageDate
dataset['CropTillageDate'] = pd.to_datetime(dataset['CropTillageDate'])
dataset_test['CropTillageDate'] = pd.to_datetime(dataset_test['CropTillageDate'])
dataset['Harv_date_CropTillageDate'] = (dataset['Harv_date'] - dataset['CropTillageDate']).dt.days
dataset_test['Harv_date_CropTillageDate'] = (dataset_test['Harv_date'] - dataset_test['CropTillageDate']).dt.days

In [26]:
# Threshing_date - Harvest_date
dataset['Threshing_date'] = pd.to_datetime(dataset['Threshing_date'])
dataset['Threshing_date_Harv_date'] = (dataset['Threshing_date'] - dataset['Harv_date']).dt.days
dataset_test['Threshing_date'] = pd.to_datetime(dataset_test['Threshing_date'])
dataset_test['Threshing_date_Harv_date'] = (dataset_test['Threshing_date'] - dataset_test['Harv_date']).dt.days

In [27]:
# Generate the month features, this will be transfered into one-hot encoding later
dataset['Harv_date_Month'] = dataset['Harv_date'].dt.month
dataset_test['Harv_date_Month'] = dataset_test['Harv_date'].dt.month

In [28]:
# Create a dictionary to map the month values to the season values
month_to_season = {1: 1, 2: 1, 3: 1, 4: 2, 5: 2,
                     6: 2, 7: 3, 8: 3, 9: 3, 10: 4,
                     11: 4, 12: 4}

In [29]:
# Map the month values to the season values
dataset['Harv_date_Season'] = dataset['Harv_date_Month'].map(month_to_season)
dataset_test['Harv_date_Season'] = dataset_test['Harv_date_Month'].map(month_to_season)

In [30]:
dataset = dataset.drop(columns=['Harv_date_Month'], axis=1)
dataset_test = dataset_test.drop(columns=['Harv_date_Month'], axis=1)

In [31]:
# drop the original date columns
dataset = dataset.drop(columns=date_columns, axis=1)
dataset_test = dataset_test.drop(columns=date_columns, axis=1)

In [32]:
dataset['Org_Crop'] = dataset['OrgFertilizers'].astype(str) + " + " + dataset['CropbasalFerts'].astype(str)
dataset_test['Org_Crop'] = dataset_test['OrgFertilizers'].astype(str) + "+" + dataset_test['CropbasalFerts'].astype(str)
dataset['Crop_FirstTop'] = dataset['CropbasalFerts'].astype(str) + "+" + dataset['FirstTopDressFert'].astype(str)
dataset_test['Crop_FirstTop'] = dataset_test['CropbasalFerts'].astype(str) + "+" + dataset_test['FirstTopDressFert'].astype(str)
dataset['Org_FirstTop'] = dataset['OrgFertilizers'].astype(str) + "+" + dataset['FirstTopDressFert'].astype(str)
dataset_test['Org_FirstTop'] = dataset_test['OrgFertilizers'].astype(str) + "+" + dataset_test['FirstTopDressFert'].astype(str)
dataset['PCropSolidOrgFertAppMethod'] = dataset['PCropSolidOrgFertAppMethod'].fillna('other')
dataset_test['PCropSolidOrgFertAppMethod'] = dataset_test['PCropSolidOrgFertAppMethod'].fillna('other')

In [33]:
# drop all those new columns
dataset = dataset.drop(columns=['Org_Crop', 'Crop_FirstTop', 'Org_FirstTop', 'PCropSolidOrgFertAppMethod'], axis=1)
dataset_test = dataset_test.drop(columns=['Org_Crop', 'Crop_FirstTop', 'Org_FirstTop', 'PCropSolidOrgFertAppMethod'], axis=1)

In [34]:
for col in dataset.columns:
    if dataset[col].dtype == 'object':
        dataset[col] = dataset[col].astype('category').cat.codes

for col in dataset_test.columns:
    if dataset_test[col].dtype == 'object':
        dataset_test[col] = dataset_test[col].astype('category').cat.codes

In [35]:
X = dataset
y = train_labels    

# Train Test split
# X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.1, random_state=200)

# print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

# Select the object columns and categorical columns
object_cols = X.select_dtypes(include=['object'])
categorical_cols = X.select_dtypes(include=['category'])
whole_categorical_cols = pd.concat([object_cols, categorical_cols], axis=1) 
# generate a list with the name of the categorical columns
whole_categorical_cols = whole_categorical_cols.columns
# Transfer into a list
whole_categorical_cols = list(whole_categorical_cols)

In [53]:
# Parameters
n_splits = 100
kf = KFold(n_splits=n_splits, shuffle=True, random_state=90)

# Prepare an array to store the RMSE for each fold
rmse_scores = []
models = []

# Initialize an empty array to hold feature importances
feature_importances = np.zeros(X.shape[1])

# Start the K-Fold cross-validation loop
for fold, (train_index, val_index) in enumerate(kf.split(X)):
    # Split the data
    X_train_fold, X_val_fold = X.iloc[train_index], X.iloc[val_index]
    y_train_fold, y_val_fold = y.iloc[train_index], y.iloc[val_index]

    # Create a LGBMRegressor object
    lgbm_model = lgb.LGBMRegressor(objective='regression', num_leaves=31, learning_rate=0.01, n_estimators=100)
    
    # Train the model
    lgbm_model.fit(
        X_train_fold, y_train_fold, 
        eval_set=[(X_val_fold, y_val_fold)], 
        eval_metric='mae', 
        categorical_feature=whole_categorical_cols
    )
    
    # Predict on the validation set
    y_pred_val = lgbm_model.predict(X_val_fold, num_iteration=0)

    # Calculate and print RMSE for the current fold
    fold_rmse = np.sqrt(mean_squared_error(y_val_fold, y_pred_val))
    rmse_scores.append(fold_rmse)
    print(f"Fold {fold}: RMSE: {fold_rmse}")

    # Accumulate feature importances
    feature_importances += lgbm_model.feature_importances_

    models.append(lgbm_model)

# After cross-validation, print the mean RMSE
print(f"Mean RMSE: {np.mean(rmse_scores)}")

# Feature importances from all folds
feature_importances = feature_importances / n_splits
# print(feature_importances)

# Create a dataframe of feature importances
# feature_importances_df = pd.DataFrame({'feature': list(X_train.columns), 'importance': feature_importances}).sort_values('importance', ascending=False)

# Print the top 20 features with their importance values
# print(feature_importances_df.head(20))
test_predictions = []

for model in models:
    # Make predictions
    fold_preds = model.predict(dataset_test, num_iteration=model.best_iteration_)
    test_predictions.append(fold_preds)

# Average these predictions
test_predictions = np.column_stack(test_predictions)
y_pred_test = np.mean(test_predictions, axis=1)

submission_df = pd.DataFrame({'ID': ID, 'Yield': y_pred_test})
submission_df.to_csv('result_file/submission_11_22_01.csv', index=False)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000353 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1259
[LightGBM] [Info] Number of data points in the train set: 3831, number of used features: 39
[LightGBM] [Info] Start training from score 594.581310
Fold 0: RMSE: 319.958615204726
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000628 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1255
[LightGBM] [Info] Number of data points in the train set: 3831, number of used features: 39
[LightGBM] [Info] Start training from score 593.968155
Fold 1: RMSE: 340.9366266161907
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000270 seconds.
You can set `force_

In [ ]:
print(set(dataset_test.columns) - set(X.columns))

In [ ]:
# Initialize a new wandb run
wandb.init(project='zindi-crop-challenge', 
           entity="lmu-seminar")

# Define your hyperparameters
hyperparameters = dict(
    objective='regression',
    num_leaves=31,
    learning_rate=0.009,
    n_estimators=120,
    # Add other hyperparameters you want to track here
)

# Save hyperparameters to wandb
wandb.config.update(hyperparameters)

# Rest of your code for preparing the data
# ...

# Parameters for cross-validation
n_splits = 5
kf = KFold(n_splits=n_splits, shuffle=True, random_state=82)

rmse_scores = []
models = []

feature_importances = np.zeros(X_train.shape[1])

# Start the K-Fold cross-validation loop
for fold, (train_index, val_index) in enumerate(kf.split(X_train)):
    X_train_fold, X_val_fold = X_train.iloc[train_index], X_train.iloc[val_index]
    y_train_fold, y_val_fold = y_train.iloc[train_index], y_train.iloc[val_index]

    # Create a LGBMRegressor object with hyperparameters from wandb
    lgbm_model = lgb.LGBMRegressor(**wandb.config)
    
    # Train the model
    lgbm_model.fit(
        X_train_fold, y_train_fold, 
        eval_set=[(X_val_fold, y_val_fold)], 
        eval_metric='mae', 
        categorical_feature=whole_categorical_cols,
        callbacks=[wandb.callback()]
    )
    
    # Predict on the validation set
    y_pred_val = lgbm_model.predict(X_val_fold, num_iteration=lgbm_model.best_iteration_)

    # Calculate RMSE and log the metric to wandb
    fold_rmse = np.sqrt(mean_squared_error(y_val_fold, y_pred_val))
    rmse_scores.append(fold_rmse)
    wandb.log({'fold_rmse': fold_rmse, 'fold': fold})

    # Feature importances
    feature_importances += lgbm_model.feature_importances_

    # Save the model to wandb
    wandb.sklearn.log_model(lgbm_model, 'LGBMRegressor')
    
    models.append(lgbm_model)

# Log mean and standard deviation of RMSE across folds to wandb
wandb.log({'mean_rmse': np.mean(rmse_scores), 'std_rmse': np.std(rmse_scores)})

# Feature importances
feature_importances = feature_importances / n_splits
# You can also log feature importances to wandb here

# Finalize and close your wandb run
wandb.finish()